In [9]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
sites = list(pd.unique(spacex_df['Launch Site']))

slider_ticks = list(range(0, 10001, 1000))
slider_marks = {}
for i in slider_ticks:
    slider_marks[i] = str(i)

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id = 'site-dropdown', 
                                             options = [
                                                 {'label': 'All Sites', 'value': 'ALL'},
                                                 {'label': sites[0], 'value': sites[0]},
                                                 {'label': sites[1], 'value': sites[1]},
                                                 {'label': sites[2], 'value': sites[2]},
                                                 {'label': sites[3], 'value': sites[3]},
                                             ],
                                             value = 'ALL',
                                             placeholder = 'Choose Site',
                                             searchable = True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id = 'success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id = 'payload-slider', 
                                                min = 0, max = 10000, step = 1000,
                                                marks = slider_marks,
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id = 'success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id = 'success-pie-chart', component_property='figure'),
               Input(component_id = 'site-dropdown', component_property = 'value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values = 'class', names = 'Launch Site', title = 'Total Success Launches By Site')
        return fig
    else:
        site = entered_site
        site_df = spacex_df[spacex_df['Launch Site'] == site]
        fig = px.pie(site_df, names = 'class', title = f'Total Success Launches for site {site}')
        return fig
    
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id = 'success-payload-scatter-chart', component_property='figure'),
              Input(component_id = 'site-dropdown', component_property = 'value'),
              Input(component_id = 'payload-slider', component_property = 'value')
             )
def get_scatter(entered_site, slider_values):
    min_val = slider_values[0]
    max_val = slider_values[1]
    if entered_site == 'ALL':
        filtered_mass_df = spacex_df[(spacex_df['Payload Mass (kg)'] > min_val) & (spacex_df['Payload Mass (kg)'] < max_val)]
        fig = px.scatter(filtered_mass_df, x = 'Payload Mass (kg)', y = 'class', color = 'Booster Version Category', title = 'Correlation between Payload and Success for all Sites')
        return fig
    else:
        filtered_mass_df = spacex_df[(spacex_df['Payload Mass (kg)'] > min_val) & (spacex_df['Payload Mass (kg)'] < max_val) & (spacex_df['Launch Site'] == entered_site)]
        fig = px.scatter(filtered_mass_df, x = 'Payload Mass (kg)', y = 'class', color = 'Booster Version Category', title = f'Correlation between Payload and Success for Site {entered_site}')
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()
